In [1]:
import tensorflow as tf

from data_loader.ISPRS_data_loader import ISPRSLoader
from models.fcn_base_model import FcnBaseModel
from trainers.aerial_image_trainer import AerialImageTrainer
# from models.unet_model import UNetModel
# from trainers.road_trainer import RoadTrainer

from utils.config import process_config
from utils.dirs import create_dirs
from utils.logger import Logger
from utils.utils import get_args
import matplotlib.pyplot as plt
import os 
from utils.visualize import *
import scipy.misc



json_file = "configs/fcn_ISPRS_Vaihingen_config.json"
json_file

config = process_config(json_file)
print(config)


colordict = {0:[255,0,0],
             1:[255,255,255],
             2:[0,0,255],
             3:[0,255,255],
             4:[0,255,0],
             5:[255,255,0]}

accuracy: sparse_categorical_crossentropy
batch_size: 5
checkpoint_dir: ./experiments\fcn_Vaihingen\checkpoint/
data_path: C:\Users\TT260256\My Project\NNs\ISPRS_benchmarks_datasets\Vaihingen\
exp_name: fcn_Vaihingen
final_model_dir: ./experiments\fcn_Vaihingen\saved_model/
image_size:
- 256
- 256
- 3
images_dir: ./experiments\fcn_Vaihingen\images/
learning_rate: 0.01
loss: scce
max_to_keep: 5
n_classes: 6
num_epochs: 30
num_iter_per_epoch: 1000
summary_dir: ./experiments\fcn_Vaihingen\summary/



In [2]:
data = ISPRSLoader(config)

Yeah this is the path to dataset : C:\Users\TT260256\My Project\NNs\ISPRS_benchmarks_datasets\Vaihingen\
Training set : 11 images, Validation set : 5 images
Size of normalized training images :  (5768, 256, 256, 3)
Size of normalized validation images :  (325, 256, 256, 3)
Size of normalized training masks :  (5768, 256, 256)
Size of normalized validation masks :  (325, 256, 256)


In [53]:
tf.reset_default_graph()
import numpy as np
import scipy
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:   
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.TRAINING], config.final_model_dir)    
        bl = sess.run("best_loss/best_loss:0")
        print(bl)        
#         for op in graph.get_operations():
#             print(op.name)
            
        X_test = graph.get_tensor_by_name("inputs/X:0")
        y_test = graph.get_tensor_by_name("inputs/y:0")
        pred =  graph.get_tensor_by_name("eval/pred:0")
        
        y_test_flatten = tf.reshape(y_test,(-1,256*256))
        pred_flatten = tf.reshape(pred,(-1,256*256))
        
        mean_iou,conf_mat = tf.metrics.mean_iou(y_test_flatten,pred_flatten,config.n_classes)
        sess.run(tf.local_variables_initializer())
        test = [data.get_data_element("valid_data",i) for i in range(300)]
        
        iou_list = []
        for idx,item in enumerate(test) :
            img = item[0]
            mask = item[1]
            out_pred = sess.run(pred,feed_dict={X_test : [img]})
#             plt.subplot(141)
#             plt.imshow(img)
#             plt.subplot(142)
#             plt.imshow(mask)
#             plt.subplot(143)
#             plt.imshow(out_pred[0])
#             plt.subplot(144)
#             out = visualize_output(img,out_pred[0],colordict)
            save_output(out, config.images_dir + "out{}.tif".format(idx))
            sess.run([conf_mat],feed_dict={X_test:[img],y_test:[mask]})
            iou_list.append(sess.run(mean_iou,feed_dict={X_test:[img],y_test:[mask]}))
        
        print("mean IoU :" ,sess.run(tf.reduce_mean(iou_list)))
            

INFO:tensorflow:Restoring parameters from ./experiments\fcn_Vaihingen\saved_model/variables\variables
0.566705
mean IoU : 0.48466
